In [29]:
import pandas as pd
import os
from utils import *

In [30]:
from dotenv import load_dotenv
load_dotenv()


True

In [31]:
token = os.getenv('data_access_token')
client_id = int(os.getenv('client_id'))
business_timezone_offset = int(os.getenv('business_timezone_offset'))
business_name = os.getenv("business_name")



In [58]:
#read all data
orders_header = pd.read_csv(f'../../data/{client_id}/raw/order_header.csv')
orders_details = pd.read_csv(f'../../data/{client_id}/raw/order_details.csv')
products = pd.read_csv(f'../../data/{client_id}/raw/products.csv')
branches = pd.read_csv(f'../../data/{client_id}/raw/branches.csv')
categories = pd.read_csv(f'../../data/{client_id}/raw/categories.csv')
options = pd.read_csv(f'../../data/{client_id}/raw/options.csv')

# Orders Headers

In [33]:
orders_header.head()

,id,client_id,branch_id,source,type,status,total_price,created_at,updated_at,ordered_at
0,0000220d-694e-466e-9e1e-763fba746540,2,964fde3e-a16d-4136-b433-f81d2c1d9b12,1,2,4,2.00,2022-11-04 05:43:37,2022-11-04 05:43:37,2022-11-04 08:43:37
1,000035d9-e082-432c-934d-040985070e32,2,96fecd4d-69bd-4669-aeef-a4724c147e9b,1,2,4,1.00,2023-01-10 07:49:43,2023-01-10 07:49:43,2023-01-10 10:49:43
2,00006231-a6fd-4b53-a0d8-14ee300f190c,2,964fde3e-a16d-4136-b433-f81d2c1d9b12,1,2,4,2.00,2022-11-15 19:34:16,2022-11-15 19:34:16,2022-11-15 22:34:16
3,000062f8-a0a7-4266-837a-080c1bc6d4eb,2,964fde3e-abf8-476c-b139-cc314022c24f,1,2,4,3.75,2022-08-05 08:16:47,2022-08-05 08:16:47,2022-08-05 11:16:47
4,0000969f-8a5b-4e6c-baa9-0928e1f84519,2,97c3bc91-eba9-4732-be73-ca9c80081518,1,2,4,2.00,2023-04-22 13:44:04,2023-04-22 13:44:04,2023-04-22 16:44:04


In [34]:
orders_types = {1:'Dine In', 2:'Pick Up', 3:'Delivery', '4':'Drive Thru'}
orders_sources = {1:'Cashier', 2:'API', 3:'Call Center'}
orders_statuses = {1:'Pending', 2:'Active', 3:'Declined', 4:'Closed', 5:'Returned', 6:'Joined', 7:'Void'}

#---------------

orders_header['type'] = orders_header['type'].map(orders_types)
orders_header['source'] = orders_header['source'].map(orders_sources)
orders_header['status'] = orders_header['status'].map(orders_statuses)


In [35]:
orders_header['ordered_at'] = pd.to_datetime(orders_header['ordered_at'])


In [36]:
orders_header.drop_duplicates(inplace=True)

In [37]:
orders_header.isnull().sum()

id             0
client_id      0
branch_id      0
source         0
type           0
status         0
total_price    0
created_at     0
updated_at     0
ordered_at     0
dtype: int64

In [38]:
orders_header.head()

,id,client_id,branch_id,source,type,status,total_price,created_at,updated_at,ordered_at
0,0000220d-694e-466e-9e1e-763fba746540,2,964fde3e-a16d-4136-b433-f81d2c1d9b12,Cashier,Pick Up,Closed,2.00,2022-11-04 05:43:37,2022-11-04 05:43:37,2022-11-04 08:43:37
1,000035d9-e082-432c-934d-040985070e32,2,96fecd4d-69bd-4669-aeef-a4724c147e9b,Cashier,Pick Up,Closed,1.00,2023-01-10 07:49:43,2023-01-10 07:49:43,2023-01-10 10:49:43
2,00006231-a6fd-4b53-a0d8-14ee300f190c,2,964fde3e-a16d-4136-b433-f81d2c1d9b12,Cashier,Pick Up,Closed,2.00,2022-11-15 19:34:16,2022-11-15 19:34:16,2022-11-15 22:34:16
3,000062f8-a0a7-4266-837a-080c1bc6d4eb,2,964fde3e-abf8-476c-b139-cc314022c24f,Cashier,Pick Up,Closed,3.75,2022-08-05 08:16:47,2022-08-05 08:16:47,2022-08-05 11:16:47
4,0000969f-8a5b-4e6c-baa9-0928e1f84519,2,97c3bc91-eba9-4732-be73-ca9c80081518,Cashier,Pick Up,Closed,2.00,2023-04-22 13:44:04,2023-04-22 13:44:04,2023-04-22 16:44:04


In [39]:
orders_header['status'].fillna('Void', inplace=True)

# Order Details

In [40]:
orders_details.head()


,id,header_id,product_id,category_id,client_id,quantity,price,created_at,updated_at
0,59d8a996-e13c-494d-9c8d-9b6fdfc0f230,0000220d-694e-466e-9e1e-763fba746540,97164374-f596-4597-b128-dc5530e995ec,97161ba2-d907-4bb1-bb82-a22c9acbcb93,2,1,2.00,2022-11-04 05:43:37,2022-11-04 05:43:37
1,bedfafdd-4194-41b4-a494-ad2264c10cc3,000035d9-e082-432c-934d-040985070e32,97164376-e22a-4073-8096-3e40dbd5c1b7,97161ba2-de90-4126-85dc-0d41ce1a266a,2,1,1.00,2023-01-10 07:49:43,2023-01-10 07:49:43
2,c35b357d-414f-430d-a032-79a111046428,00006231-a6fd-4b53-a0d8-14ee300f190c,97164374-f592-44e8-8089-242fef28e599,97161ba2-d907-4bb1-bb82-a22c9acbcb93,2,1,2.00,2022-11-15 19:34:16,2022-11-15 19:34:16
3,0ea512d3-15f5-4f6c-acc6-d273bd1a08d8,000062f8-a0a7-4266-837a-080c1bc6d4eb,964fde67-6f51-4870-8003-e1cfaa0b3ccb,964fde5c-05f2-4ed5-a55e-882453ad5311,2,1,2.50,2022-08-05 08:16:47,2022-08-05 08:16:47
4,1e4e0368-5124-443f-8479-d7fa1dd4954f,000062f8-a0a7-4266-837a-080c1bc6d4eb,964fde6f-82e4-43e8-96a7-d39cd5a922d4,964fde5c-05f2-4ed5-a55e-882453ad5311,2,1,1.25,2022-08-05 08:16:47,2022-08-05 08:16:47


# Products

In [41]:
products.head()

,id,client_id,sku,slug,name,category_id,is_active,is_stock,price,created_at,updated_at,deleted_at
0,964fde65-52d5-491c-b227-da1603227e82,2,MER-013,coffee-bean-250g-delivery-f,Coffee Bean 250g (Delivery) - F,96ecc1b0-0590-4e3d-9325-91b371c2a18a,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:18,2022-08-22 21:36:18
1,964fde65-55c7-4cc3-84df-285efec97d70,2,MER-015,matte-mug-delivery-f,Matte Mug (Delivery) - F,96ecc1b0-0590-4e3d-9325-91b371c2a18a,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19
2,964fde65-57c6-450f-b537-38b8a3395ee0,2,sk-004,hot-chocolate-f-hot-chocolate-f,Hot chocolate-F (Hot chocolate-F),964fde5b-9a21-44db-9dab-b98ff6a4f3d0,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19
3,964fde65-5969-4b44-9a47-00849d643cb8,2,sk-007,calibration-calibration,Calibration (Calibration),964fde5b-55b6-4d27-af90-22bd41e2f54e,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19
4,964fde65-5b14-4fee-a63d-757f5a46dce0,2,sk-005,iced-lotus-latte-f-super,Iced Lotus Latte-F (SUPER),964fde5b-9a21-44db-9dab-b98ff6a4f3d0,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19


In [42]:
products.name.unique().tolist()

['Coffee Bean 250g (Delivery) - F',
 'Matte Mug (Delivery) - F',
 'Hot chocolate-F (Hot chocolate-F)',
 'Calibration (Calibration)',
 'Iced Lotus Latte-F (SUPER)',
 'Hot Spanish latte (Delivery)',
 'Iced Pistachio Latte (Delivery) (SUPER)',
 'Fatireh ZC - F',
 'Cappuccino (Cappuccino)',
 'Hot Spanish latte',
 'Qashabeyyeh (Qashabeyyeh)',
 'Iced Latte (REGULAR Iced Latte)',
 'Extra Pistachio (Extra Pistachio)',
 'Hot Pistachio Latte (Delivery)',
 'Tiramisu Single Pc (Tiramisu Single Pc)',
 "Mini Nutella Ka'ek-F",
 'Iced Americano-F (SUPER)',
 'Tiramisu Tray Delivery (Tiramisu Tray Delivery)',
 'Iced Americano (SUPER Iced Americano)',
 "Cheese Ka'ek (Ka'ek Ka'eek)",
 'Hot Caramel Latte (Delivery)',
 "Mini Cheese Ka'ek",
 "Turkey Cheese Ka'ek (Delivery) - F",
 'Iced Rose latte-F (SUPER)',
 'Kashtaban Jeans (Kashtaban Jeans)',
 'Extra Hazelnut - F (Extra Hazelnut - F)',
 'Hot Lotus Latte-F',
 'Matte mug (Matte mug)',
 'Iced Spanish latte (Delivery)-F (REGULAR)',
 'Extra Pistachio - F (Extr

In [43]:
products['slug'] = products.name.apply(lambda x: generate_slug(x))


In [44]:
products.head()

,id,client_id,sku,slug,name,category_id,is_active,is_stock,price,created_at,updated_at,deleted_at
0,964fde65-52d5-491c-b227-da1603227e82,2,MER-013,coffee-bean-250g-delivery-f,Coffee Bean 250g (Delivery) - F,96ecc1b0-0590-4e3d-9325-91b371c2a18a,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:18,2022-08-22 21:36:18
1,964fde65-55c7-4cc3-84df-285efec97d70,2,MER-015,matte-mug-delivery-f,Matte Mug (Delivery) - F,96ecc1b0-0590-4e3d-9325-91b371c2a18a,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19
2,964fde65-57c6-450f-b537-38b8a3395ee0,2,sk-004,hot-chocolate-f-hot-chocolate-f,Hot chocolate-F (Hot chocolate-F),964fde5b-9a21-44db-9dab-b98ff6a4f3d0,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19
3,964fde65-5969-4b44-9a47-00849d643cb8,2,sk-007,calibration-calibration,Calibration (Calibration),964fde5b-55b6-4d27-af90-22bd41e2f54e,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19
4,964fde65-5b14-4fee-a63d-757f5a46dce0,2,sk-005,iced-lotus-latte-f-super,Iced Lotus Latte-F (SUPER),964fde5b-9a21-44db-9dab-b98ff6a4f3d0,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19


In [45]:
products.sku.unique().tolist()

['MER-013',
 'MER-015',
 'sk-004',
 'sk-007',
 'sk-005',
 'sk-008',
 'sk-010',
 'FOD-027',
 'sk-009',
 'sk-012',
 'MER-006',
 'sk-023',
 'sk-015',
 'sk-016',
 'FOD-034',
 'FOD-017',
 'sk-019',
 'FOD-036',
 'sk-033',
 'sk-032',
 'sk-028',
 'FOD-013',
 'FOD-033',
 'sk-029',
 'MER-010',
 'sk-039',
 'sk-036',
 'MER-001',
 'sk-052',
 'sk-038',
 'sk-037',
 'sk-011',
 'FOD-028',
 'sk-013',
 'sk-040',
 'sk-021',
 'sk-030',
 'sk-018',
 'sk-050',
 'sk-024',
 'FOD-011',
 'FOD-012',
 'sk-022',
 'sk-045',
 'sk-017',
 'sk-014',
 'sk-047',
 'sk-060',
 'FOD-006',
 'FOD-016',
 'sk-079',
 'sk-067',
 'sk-043',
 'sk-083',
 'sk-035',
 'FOD-015',
 'sk-046',
 'sk-049',
 'sk-058',
 'FOD-014',
 'sk-042',
 'sk-055',
 'sk-063',
 'sk-100',
 'sk-085',
 'sk-062',
 'FOD-021',
 'FOD-037',
 'sk-075',
 'sk-056',
 'sk-073',
 'FOD-004',
 'sk-074',
 'sk-102',
 'sk-059',
 'sk-092',
 'sk-096',
 'sk-101',
 'MER-012',
 'sk-080',
 'sk-095',
 'sk-086',
 'FOD-001',
 'sk-081',
 'FOD-002',
 'sk-098',
 'sk-077',
 'sk-057',
 'sk-104

In [46]:
products[products.name.isnull()]

,id,client_id,sku,slug,name,category_id,is_active,is_stock,price,created_at,updated_at,deleted_at


In [47]:
products.sku.nunique()

355

# Branches

In [48]:
branches

,id,client_id,name,slug,opening_from,opening_to,created_at,updated_at,deleted_at
0,964fde3e-a16d-4136-b433-f81d2c1d9b12,2,Abdali Boulevard,abdali-boulevard,04:00,04:00,2022-05-16 10:42:32,2023-01-16 09:15:16,NaN
1,964fde3e-a371-426e-a209-d6f3ff79d2a1,2,Sports City,sports-city,04:00,04:00,2022-05-16 10:42:32,2023-01-16 09:16:20,NaN
2,964fde3e-a88f-4426-b476-62d018d33feb,2,Abdoun,abdoun,04:00,04:00,2022-05-16 10:42:32,2023-01-16 09:17:01,NaN
3,964fde3e-aa4d-48aa-8c29-e0c8c77cf0b6,2,Fuhais,fuhais,04:00,04:00,2022-05-16 10:42:32,2023-01-16 09:17:23,NaN
4,964fde3e-abf8-476c-b139-cc314022c24f,2,7th Circle,7th-circle,04:00,04:00,2022-05-16 10:42:32,2023-05-02 11:05:39,NaN
5,964fde3e-aec3-41f6-8fc9-b201a919e1dd,2,Events Backup (Previously Main Warehouse),events-backup-previously-main-warehouse,04:00,04:00,2022-05-16 10:42:32,2023-03-22 13:36:05,NaN
6,96ed24b8-517e-465f-b29c-ea84ec013766,2,Kia Showroom,kia-showroom,04:00,04:00,2022-08-02 15:04:34,2023-01-16 09:14:49,NaN
7,96fecd4d-69bd-4669-aeef-a4724c147e9b,2,Event Branch,event-branch,04:00,04:00,2022-08-11 09:45:02,2023-01-16 09:08:03,NaN
8,97c3bc91-eba9-4732-be73-ca9c80081518,2,Zarqa - Karama,zarqa-karama,04:00,04:00,2022-11-17 07:16:20,2023-01-16 09:13:56,NaN


In [49]:
branches['opening_from'] = pd.to_datetime(branches['opening_from'])
branches['opening_to'] = pd.to_datetime(branches['opening_to'])

# branches['opening_from'] = branches['opening_from'].dt.hour
# branches['opening_to'] = branches['opening_to'].dt.hour

In [50]:
branches

,id,client_id,name,slug,opening_from,opening_to,created_at,updated_at,deleted_at
0,964fde3e-a16d-4136-b433-f81d2c1d9b12,2,Abdali Boulevard,abdali-boulevard,2023-06-15 04:00:00,2023-06-15 04:00:00,2022-05-16 10:42:32,2023-01-16 09:15:16,NaN
1,964fde3e-a371-426e-a209-d6f3ff79d2a1,2,Sports City,sports-city,2023-06-15 04:00:00,2023-06-15 04:00:00,2022-05-16 10:42:32,2023-01-16 09:16:20,NaN
2,964fde3e-a88f-4426-b476-62d018d33feb,2,Abdoun,abdoun,2023-06-15 04:00:00,2023-06-15 04:00:00,2022-05-16 10:42:32,2023-01-16 09:17:01,NaN
3,964fde3e-aa4d-48aa-8c29-e0c8c77cf0b6,2,Fuhais,fuhais,2023-06-15 04:00:00,2023-06-15 04:00:00,2022-05-16 10:42:32,2023-01-16 09:17:23,NaN
4,964fde3e-abf8-476c-b139-cc314022c24f,2,7th Circle,7th-circle,2023-06-15 04:00:00,2023-06-15 04:00:00,2022-05-16 10:42:32,2023-05-02 11:05:39,NaN
5,964fde3e-aec3-41f6-8fc9-b201a919e1dd,2,Events Backup (Previously Main Warehouse),events-backup-previously-main-warehouse,2023-06-15 04:00:00,2023-06-15 04:00:00,2022-05-16 10:42:32,2023-03-22 13:36:05,NaN
6,96ed24b8-517e-465f-b29c-ea84ec013766,2,Kia Showroom,kia-showroom,2023-06-15 04:00:00,2023-06-15 04:00:00,2022-08-02 15:04:34,2023-01-16 09:14:49,NaN
7,96fecd4d-69bd-4669-aeef-a4724c147e9b,2,Event Branch,event-branch,2023-06-15 04:00:00,2023-06-15 04:00:00,2022-08-11 09:45:02,2023-01-16 09:08:03,NaN
8,97c3bc91-eba9-4732-be73-ca9c80081518,2,Zarqa - Karama,zarqa-karama,2023-06-15 04:00:00,2023-06-15 04:00:00,2022-11-17 07:16:20,2023-01-16 09:13:56,NaN


# Categories

In [51]:
categories

,id,client_id,name,slug,created_at,updated_at,deleted_at
0,97161ba2-d56a-4a37-a75f-1303a0c4a15c,2,Food,food,2022-08-22 23:48:07,2023-03-15 20:16:26,NaN
1,97161ba2-d907-4bb1-bb82-a22c9acbcb93,2,Sweet coffee,sweet-coffee,2022-08-22 23:48:07,2023-03-15 20:16:37,NaN
2,97161ba2-da3c-4f23-8a52-218be3fb5fd7,2,Delivery food,delivery-food,2022-08-22 23:48:07,2023-03-15 20:16:53,NaN
3,97161ba2-da47-449e-a2ff-c16deafb5c91,2,Milk coffee,milk-coffee,2022-08-22 23:48:07,2022-08-22 23:48:07,NaN
4,97161ba2-dc03-4f8b-a35d-f1cc3d809906,2,Bottles,bottles,2022-08-22 23:48:07,2023-03-15 20:39:45,NaN
5,97161ba2-de90-4126-85dc-0d41ce1a266a,2,Black coffee,black-coffee,2022-08-22 23:48:07,2023-03-15 20:35:35,NaN
6,97161ba2-df3b-47d3-a02e-8f950ae7033f,2,Delivery drinks,delivery-drinks,2022-08-22 23:48:07,2022-08-22 23:48:07,NaN
7,97161ba2-e1ed-4711-8d43-3f0e71acc13c,2,Merchandise,merchandise,2022-08-22 23:48:07,2023-03-13 09:42:18,NaN
8,97161ba2-e228-4a6d-8d01-10e35ed3ec22,2,Other,other,2022-08-22 23:48:07,2022-08-22 23:48:07,NaN
9,98ad4709-821b-4bb6-8c8e-0157f7510d20,2,Delivery merch,delivery-merch,2023-03-13 09:31:25,2023-03-15 20:39:36,NaN


# Order Options (Modifiers)

In [59]:
options.head()

,order_details_id,option_id,option_name,option_name_localized,option_sku,option_quantity,option_partition,option_unit_price,option_total_price,option_total_cost
0,59d8a996-e13c-494d-9c8d-9b6fdfc0f230,97a9749c-e3c1-4ff6-b190-690a3a0c1814,Regular,عادي,option-modifier-9924,1,1,2.0,2.0,0.43524
1,59d8a996-e13c-494d-9c8d-9b6fdfc0f230,97a9749c-e58b-404b-9676-7aa86f54f967,Full Fat,حليب كامل الدسم,option-modifier-9882,1,1,0.0,0.0,0.05333
2,bedfafdd-4194-41b4-a494-ad2264c10cc3,983069a5-e226-4ec9-97f3-a78be49d0ff4,Regular,عادي,option-modifier-9868,1,1,1.0,1.0,0.19303
3,c35b357d-414f-430d-a032-79a111046428,97c0be81-4a84-4192-8adf-39944db65dcd,Regular,عادي,option-modifier-9933,1,1,2.0,2.0,0.49796
4,c35b357d-414f-430d-a032-79a111046428,97c0be81-4ceb-48b4-8872-1a01799e0e31,Full Fat,حليب كامل الدسم,option-modifier-9882,1,1,0.0,0.0,0.05333


# Write

In [52]:
orders_header.to_csv(f'../../data/{client_id}/processed/order_header.csv', index=False)

In [53]:
orders_details.to_csv(f'../../data/{client_id}/processed/order_details.csv', index=False)

In [54]:
products.to_csv(f'../../data/{client_id}/processed/products.csv', index=False)


In [55]:
branches.to_csv(f'../../data/{client_id}/processed/branches.csv', index=False)

In [56]:
categories.to_csv(f'../../data/{client_id}/processed/categories.csv', index=False)

In [ ]:
options.to_csv(f'../../data/{client_id}/processed/options.csv', index=False)